In [2]:
import pandas as pd
from pypdf import PdfReader
import re
from datetime import datetime
import os

In [ ]:
date_pattern_year = r'^\d{2}/\d{2}/\d{4}'
date_pattern = r'^\d{2}/\d{2}'
mistaken_pattern = r'(\d+/\d+)\s*([\d,.]+)'
mistaken_pattern1 = r'^([a-zA-Z]+)\\S([0-9]+\\,[0-9]+)',
fraction_pattern = r'(\d/\d)'
banrisul_pattern = r'\+\+\s*MOVIMENTOS\s+(\w+)\/(\d{4})'
banrisul_pattern_2 = r'^(\d*)\s+(\D+)\s+\d+\s+(\d+\,\d+)(\-*)'

In [3]:
def extract_data(file_path,passwd=None):
    reader = PdfReader(file_path)
    if reader.is_encrypted:
        reader.decrypt(passwd)
    return [page.extract_text().splitlines() for page in reader.pages]
        

In [4]:
file = '/home/jpchagas/Downloads/guaiaca_dash/nubank/nubank_fatura_janeiro.pdf'

In [7]:
charges = extract_data(file)

In [9]:
charges

[[' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  'Olá, Rafaela!',
  'Esta é a sua fatura de',
  'janeiro, no valor de',
  'R$ 222,29',
  'Data do vencimento: 02 JAN 2024 ',
  'Limite total: R$ 8.950,00 ',
  'Valor máximo para transações: ',
  'Saque no crédito: R$ 1.342,50 ',
  'Pix no crédito: R$ 8.950,00 ',
  'Pagamento de boletos no crédito: R$ 8.950,00 ',
  ' ',
  ' ',
  'Pagamento mínimo para ',
  'não ficar em atraso ',
  ' ',
  'R$ 33,34 ',
  ' ',
  'Ao fazer o pagamento mínimo, o saldo restante ',
  'vai entrar no rotativo e o valor de R$ 220,77 será ',
  'adicionado à sua próxima fatura, já inclusos ',
  'R$ 31,83 de juros e IOF. ',
  'Saiba mais ',
  ' ',
  ' 1 de 4'],
 ['Confira as opções de pagamento e ',
  'entenda o que cabe no seu bolso. ',
  '1. Pagar o valor total da fatura ',
  '  Essa é sempre a melhor escolha ',
  'Com o pagamento total, você quita o saldo da sua fatura, libera o limite de crédito do ',
  'seu cartão e não paga juros ou multa. ',
  ' ',
  'Como fazer: vo

In [13]:
charges_extracted = []
for x in range(len(charges)):
    for y in range(len(charges[x])):
        if re.match('^([0-9]+)\s(\D+)$',charges[x][y]):
            raw_data = charges[x][y].split()
            data = f'{raw_data[0]}/{get_month_number_abbr(raw_data[1].lower())}/{datetime.now().year}'
            raw_description = charges[x][y+2].split("-")
            descricao = raw_description[0]
            parcela=""
            qt_parcelas=""
            if len(raw_description)>1:
                parcela,qt_parcelas = raw_description[1].split("/")
            valor = self.sh.convert_brazil_to_us_currency(charges[x][y+3])
            charges_extracted.append((data,descricao,parcela,qt_parcelas,self.sh.convert_brazil_to_us_currency(valor)))
        df = pd.DataFrame(charges_extracted, columns =['data', 'descricao','parcela','qt_parcelas', 'valor'])
                # Step 3: Drop unwanted rows based on description
        df = df[~df['descricao'].str.contains("Pagamento em", case=False, na=False)]
        df['metodo'] = 'cc'
        df['banco'] = 'nubank'
        df['transacao'] = 'despesa'

NameError: name 'self' is not defined

In [ ]:
def extract_info(file_path,passwd=None):
    pdf_data = extract_data(file_path,passwd)
    # Combine patterns into a single pattern using alternation
    combined_pattern = re.compile(f"({date_pattern})|({date_pattern_year})|({mistaken_pattern})")
    # Use list comprehension to filter lines that match any of the combined patterns
    return [line for data in pdf_data for line in data if combined_pattern.match(line)]

In [ ]:
def convert_brazil_to_us_currency(brazil_currency):
    # Remove any dots used as thousand separators
    brazil_currency_no_thousand_sep = brazil_currency.replace('.', '')
    # Replace the comma used as a decimal separator with a dot
    us_currency = brazil_currency_no_thousand_sep.replace(',', '.')
    return us_currency

In [12]:
def get_month_number_abbr(month_name_abbr):
    # Define a mapping of month abbreviations to month numbers
    month_abbr_to_number = {
        'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4,
        'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8,
        'set': 9, 'out': 10, 'nov': 11, 'dez': 12
    }
    
    # Convert the abbreviation to lowercase to handle case-insensitivity
    month_name_abbr_lower = month_name_abbr.lower()
    
    # Lookup the month number based on the abbreviation
    month_number = month_abbr_to_number.get(month_name_abbr_lower)
    
    return month_number

In [ ]:
def extract_charges(text):
    # Split the text into lines
    lines = text.split('\n')
    
    # Initialize variables
    charges = []
    current_cardholder = None
    
    # Pattern to match cardholder names
    cardholder_pattern = re.compile(r'^\*\*(.+?)\*\*$')
    
    # Pattern to match charges
    charge_pattern = re.compile(r'^\*(\d{2}/\d{2})\s(.+?)\s-\sR\$\s([\d,]+(\(\d+/\d+\))?)\*')
    
    for line in lines:
        # Check for cardholder name
        cardholder_match = cardholder_pattern.match(line.strip())
        if cardholder_match:
            current_cardholder = cardholder_match.group(1)
            continue
        
        # Check for charges
        charge_match = charge_pattern.match(line.strip())
        if charge_match:
            date, description, amount = charge_match.groups()[:3]
            charges.append({
                'cardholder': current_cardholder,
                'date': date,
                'description': description,
                'amount': amount
            })
    
    return charges

def create_dataframe(charges):
    # Create a DataFrame from the list of charges
    df = pd.DataFrame(charges)
    return df

In [ ]:
file_path = ['/home/jpchagas/Downloads/guaiaca_dash/bradesco/bradesco_fatura_janeiro.pdf',
            '/home/jpchagas/Downloads/guaiaca_dash/bradesco/bradesco_fatura_fevereiro.pdf',
            '/home/jpchagas/Downloads/guaiaca_dash/bradesco/bradesco_fatura_marco.pdf',
            '/home/jpchagas/Downloads/guaiaca_dash/bradesco/bradesco_fatura_abril.pdf',
            '/home/jpchagas/Downloads/guaiaca_dash/bradesco/bradesco_fatura_maio.pdf']
consolidated_info = []
for file in file_path:
    charges = extract_data(file,"5593")
    consolidated_info.append(charges)
# Extract and create DataFrame
#charges = extract_charges(bill_text)
#df = create_dataframe(charges)

# Display the DataFrame
#print(df)

In [ ]:
consolidated_info

In [ ]:
parsed_data = []

# Loop through each main section
for section in consolidated_info:
    for card_section in section:
        # Split the section into lines
        lines = card_section
        card_info = {
            'Card Details': [],
            'Transactions': [],
            'Total': None
        }
        for line in lines:
            if line.startswith('XXXX.XXXX.XXXX'):
                # This line contains cardholder details
                card_info['Card Details'] = line.split('XXXX')
            elif line.startswith('.Total para'):
                # This line contains the total
                card_info['Total'] = line.split('.Total para ')[1]
            else:
                # This line contains a transaction
                card_info['Transactions'].append(line)
        parsed_data.append(card_info)

# Printing parsed data
for card in parsed_data:
    print(f"Card Details: {card['Card Details']}")
    print("Transactions:")
    for transaction in card['Transactions']:
        print(f"  - {transaction}")
    print(f"Total: {card['Total']}")
    print("")


In [ ]:
tuples = []
for c in charges:
    c_splitted = c.split()
    data = c_splitted[0]
    descricao = ""
    valor = 0
    transacao = ""
    if c_splitted[-1] == 'C':
        transacao = "receita"
        valor = c_splitted[-2]
        descricao = ''.join(c_splitted[1:-3])
        tuples.append((data,descricao,valor,transacao))
    elif c_splitted[-1] == 'D':
        transacao = "despesa"
        valor = c_splitted[-2]
        descricao = ''.join(c_splitted[1:-3])
        tuples.append((data,descricao,valor,transacao))

df = pd.DataFrame(tuples, columns =['data', 'descricao', 'valor','transacao'])
df['metodo'] = 'pix ou transferencia'
df['banco'] = 'c6'
df['parcela'] = 1
df['qt_parcelas'] = 1

In [ ]:
df['data'] = pd.to_datetime(df['data'],format='%d/%m/%Y')

In [ ]:
df['month'] = df['data'].dt.month

In [ ]:
grouped = df.groupby('month')

In [ ]:
dfs = {month: grouped.get_group(month) for month in grouped.groups}

In [ ]:
for i in dfs:
    dfs[i].to_csv(f'/home/jpchagas/Downloads/C6_{i}.csv')

In [ ]:
data = ""
descricao = ""
valor = 0
transacao = ""
month_number = ""
year = ""
tuples = []
for c in charges:
    match_data = re.match('\+\+\s*MOVIMENTOS\s+(\w+)\/(\d{4})',c)
    match_transaction = re.match('^(\d*)\s+(\D+)\s+\d+\s+(\d+\,\d+)(\-*)', c)
    if match_data:
        print
        month_number = get_month_number_abbr(match_data.group(1))
        year = match_data.group(2)
    elif match_transaction:
        if match_transaction.group(1) != '':
            data = f'{match_transaction.group(1)}/{month_number}/{year}'
        descricao = match_transaction.group(2)
        valor = match_transaction.group(3)
        if match_transaction.group(4) == '':
            transacao = "receita"
        else:
            transacao = "despesa"
        tuples.append((data,descricao,float(convert_brazil_to_us_currency(valor)),transacao))

In [ ]:
tuples

In [ ]:
df = pd.DataFrame(tuples, columns =['data', 'descricao', 'valor','transacao'])
df['metodo'] = 'pix ou transferencia'
df['banco'] = 'banrisul'
df['parcela'] = 1
df['qt_parcelas'] = 1

In [ ]:
charges_tuples=[]
for c in charges:
    c_splitted = c.split()
    data = c_splitted[0]
    description = ""
    for i in range(2,len(c_splitted)-2):
            description += c_splitted[i]
    value = c_splitted[len(c_splitted)-2]
    transaction_type = c_splitted[len(c_splitted)-1]
    charges_tuples.append((data,description,float(convert_brazil_to_us_currency(value)),transaction_type))

In [ ]:
charges_tuples

In [ ]:
charges_tuples=[]
for c in charges:
    c_splitted = c.split()
    data = None
    if re.match(date_pattern_year, c_splitted[0]):
        print("match")
        data = c_splitted[0]
    else:
        print("no match")
        current_year = datetime.now().year
        data = c_splitted[0]+"/"+str(current_year)
    amount = c_splitted[len(c_splitted)-1]
    description = ""
    fraction = ""
    if re.match(fraction_pattern, c_splitted[len(c_splitted)-2]):
        fraction = c_splitted[len(c_splitted)-2]
        for i in range(1,len(c_splitted)-2):
            description += c_splitted[i]
        #Concatenar outros elementos
        charges_tuples.append((data,description,fraction,float(convert_brazil_to_us_currency(amount))))
    else:
        for i in range(1,len(c_splitted)-1):
            description += c_splitted[i]
        #Concatenar outros elements
        charges_tuples.append((data,description,fraction,float(convert_brazil_to_us_currency(amount))))
df = pd.DataFrame(charges_tuples, columns =['data', 'descricao', 'parcela','valor'])
indexDescription1 = df[df['description'] == 'PAGTO.PORDEBEMC/C'].index
indexDescription2 = df[df['description'] == 'SALDOANTERIOR'].index
df.drop(indexDescription1 , inplace=True)
df.drop(indexDescription2 , inplace=True)
qt_parcelas = []
parcela = []
for i in df['parcela']:
    p,qt_p = i.split("/")
    parcela.append(p)
    qt_parcelas.append(qt_p)
df['parcela'] = parcela
df['qt_parcelas'] = qt_parcelas
df = df.drop(columns=['Parcela'])
df['transacao'] = 'despesa'
df['metodo'] = 'cc'
df['banco'] = 'c6'

In [ ]:
charges_tuples